In [1]:
import pandas as pd
import numpy as np
import tempfile
import os.path
import calendar

# Ouverture du dataset avec remplacements des données manquantes afin qu'elles soient detectable par pandas en tant que Na/NaN

In [2]:
missing_values = ["n/a", "na", "--", "-"]
file_path = "D:\_IMSD\Data viz'\dataset.csv""
#file_path = "/home/sam/Documents/cours/DataViz/project_dataviz2_imsd-master/dataset.csv"
dataset = pd.read_csv(file_path, encoding="ISO-8859-1", na_values = missing_values)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# copy of DataFrame => cleaned one
df = dataset.copy()
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


In [6]:
'''for i in df.columns:
    print(i)'''

'for i in df.columns:\n    print(i)'

In [7]:
'''filtered_df = df[df['addnotes'].notnull()]
filtered_df'''

"filtered_df = df[df['addnotes'].notnull()]\nfiltered_df"

# Suppression des attaques n'ayant pas de date complète

#### Suppression des lignes n'ayant pas de mois ou de jours (0.49%)

In [8]:
#df[df['imonth'] == 0].count()
#df[df['iday'] == 0].count()
print((891/181691)*100)

0.4903930299244321


In [9]:
df.drop(df[(df.imonth == 0) | (df.iday ==0)].index, inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180800 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.6+ MB


# Remplacer les numéros de mois & jours par leur abbréviation

In [11]:
df['imonth'] = df['imonth'].apply(lambda x: calendar.month_abbr[x])

In [12]:
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,Jul,2,NaN,0,NaN,58,Dominican Republic,2,Central America & Caribbean,NaN,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
5,197001010002,1970,Jan,1,NaN,0,NaN,217,United States,1,North America,Illinois,Cairo,37.005105,-89.176269,1.0,0,NaN,1/1/1970: Unknown African American assailants ...,1,1,1,0.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Cairo Police Department,Cairo Police Headquarters,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Black Nationalists,NaN,NaN,NaN,NaN,NaN,To protest the Cairo Illinois Police Deparment,0.0,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Several gunshots were fired.,0.0,0.0,0.0,0.0,0.0,0.0,1,3.0,Minor (likely < $1 million),NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Cairo Chief of Police, William Petersen, r...","""Police Chief Quits,"" Washington Post, January...","""Cairo Police Chief Quits; Decries Local 'Mili...","Christopher Hewitt, ""Political Violence and Te...",Hewitt Project,-9,-9,0,-9,NaN
6,197001020001,1970,Jan,2,NaN,0,NaN,218,Uruguay,3,South America,Montevideo,Montevideo,-34.891151,-56.187214,1.0,0,NaN,NaN,1,1,1,0.0,NaN,NaN,0.0,0,0,1,Assassination,NaN,NaN,NaN,NaN,3,Police,25.0,Police Security Forces/Officers,Uruguayan Police,Juan Maria de Lucah/Chief of Directorate of in...,218.0,Uruguay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupamaros (Uruguay),NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automatic firearm,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
7,197001020002,1970,Jan,2,NaN,0,NaN,217,United States,1,North America,California,Oakland,37.791927,-122.225906,1.0,0,Edes Substation,1/2/1970: Unkn

# Création du dataset pour le random button: conserver uniquement les lignes n'ayant aucune valeur manquante des événements à compter 1997

In [13]:
higher_1997 = df.copy()

In [14]:
data = higher_1997[higher_1997['iyear'] >= 1997]
data.head()
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117110 entries, 64310 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(21), object(59)
memory usage: 121.5+ MB


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
64310,199701010001,1997,Jan,1,NaN,0,NaN,141,Nepal,6,South Asia,Central,Bethan,27.493003,85.824033,1.0,1,NaN,NaN,1,1,1,-9.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,3,Police,22.0,"Police Building (headquarters, station, school)",Government,Police Department,141.0,Nepal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,5.0,Unknown Gun Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Firearm,5.0,NaN,NaN,2.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
64311,199701010002,1997,Jan,1,NaN,0,NaN,155,West Bank and Gaza Strip,10,Middle East & North Africa,West Bank,Hebron,31.530243,35.094162,1.0,0,NaN,NaN,1,1,1,-9.0,NaN,NaN,0.0,1,0,2,Armed Assault,NaN,NaN,NaN,NaN,14,Private Citizens & Property,74.0,Marketplace/Plaza/Square,NaN,Market Place,155.0,West Bank and Gaza Strip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jewish Extremists,Settlers,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Firearms,2.0,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M-16,0.0,NaN,NaN,7.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,NaN
64312,199701010003,1997,Jan,1,NaN,0,NaN,217,United States,1,North America,Oklahoma,Tulsa,36.149745,-95.993334,1.0,0,NaN,NaN,1,1,1,-9.0,NaN,NaN,1.0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,5,Abortion Related,40.0,Clinics,Reproductive Services,Abortion Clinic,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anti-Abortion extremists,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,Incendiary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Incendiary,0.0,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,0,-9,"199701010003, 199701010004, 199701010005"
64313,199701010004,1997,Jan,1,NaN,0,NaN,217,United States,1,North America,Oklahoma,Tulsa,36.149745,-95.993334,1.0,0,NaN,NaN,1,1,1,-9.0,NaN,NaN,1.0,1,0,7,Facility/Infrastructure Attack,NaN,NaN,NaN,NaN,5,Abortion Related,40.0,Clinics,Reproductive Services,Abortion Clinic,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anti-Abortion extremists,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [15]:
random_button = data[
    (data.iyear.notnull()) & (data.imonth.notnull()) & (data.iday.notnull()) & (data.imonth.notnull()) &
    (data.country_txt.notnull()) & (data.city.notnull()) & (data.summary.notnull()) & (data.attacktype1_txt.notnull()) &
    (data.targsubtype1_txt.notnull()) & (data.gname.notnull()) & (data.nkill.notnull())]

In [16]:
random_button.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102371 entries, 64315 to 181689
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(21), object(59)
memory usage: 106.2+ MB


# Renomer les colonnes

In [18]:
columns = [i for i in data.columns]
print(columns)

['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended', 'resolution', 'country', 'country_txt', 'region', 'region_txt', 'provstate', 'city', 'latitude', 'longitude', 'specificity', 'vicinity', 'location', 'summary', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative', 'alternative_txt', 'multiple', 'success', 'suicide', 'attacktype1', 'attacktype1_txt', 'attacktype2', 'attacktype2_txt', 'attacktype3', 'attacktype3_txt', 'targtype1', 'targtype1_txt', 'targsubtype1', 'targsubtype1_txt', 'corp1', 'target1', 'natlty1', 'natlty1_txt', 'targtype2', 'targtype2_txt', 'targsubtype2', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2', 'natlty2_txt', 'targtype3', 'targtype3_txt', 'targsubtype3', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3', 'natlty3_txt', 'gname', 'gsubname', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'motive', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'nperps', 'nperpcap', 'claimed', 'claimmode', 'claimmode_txt', 'claim2', 'claimmode2', 'claim

In [19]:
columns = [i for i in data.columns]
print(columns)

['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended', 'resolution', 'country', 'country_txt', 'region', 'region_txt', 'provstate', 'city', 'latitude', 'longitude', 'specificity', 'vicinity', 'location', 'summary', 'crit1', 'crit2', 'crit3', 'doubtterr', 'alternative', 'alternative_txt', 'multiple', 'success', 'suicide', 'attacktype1', 'attacktype1_txt', 'attacktype2', 'attacktype2_txt', 'attacktype3', 'attacktype3_txt', 'targtype1', 'targtype1_txt', 'targsubtype1', 'targsubtype1_txt', 'corp1', 'target1', 'natlty1', 'natlty1_txt', 'targtype2', 'targtype2_txt', 'targsubtype2', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2', 'natlty2_txt', 'targtype3', 'targtype3_txt', 'targsubtype3', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3', 'natlty3_txt', 'gname', 'gsubname', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'motive', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'nperps', 'nperpcap', 'claimed', 'claimmode', 'claimmode_txt', 'claim2', 'claimmode2', 'claim

In [20]:
data.rename(columns={'iyear':'year','imonth':'month','iday':'day','country_txt':'country','region_txt':'region',
                     'crit1':'criteria','attacktype1_txt':'attacktype','targtype1_txt':'targtype',
                     'targsubtype1_txt':'targsubtype','corp1':'targeted_entity','natlty1_txt':'targeted_nationality',
                     'gname':'perpetrator_group','gsubname':'perpetrator_details','claimmode_txt':'claim_mode',
                     'compclaim':'competing_claim','weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype',
                     'weapdetail':'weapon_details','nwound':'nb_vitcims_injured','nwoundte':'nb_perpetrators_injured',
                     'propextent_txt':'property_damage','ransomamt':'amount_ransom','hostkidoutcome_txt':'hostkidoutcome'
                    },inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [21]:
df.rename(columns={'iyear':'year','imonth':'month','iday':'day','country_txt':'country','region_txt':'region',
                     'crit1':'criteria','attacktype1_txt':'attacktype','targtype1_txt':'targtype',
                     'targsubtype1_txt':'targsubtype','corp1':'targeted_entity','natlty1_txt':'targeted_nationality',
                     'gname':'perpetrator_group','gsubname':'perpetrator_details','claimmode_txt':'claim_mode',
                     'compclaim':'competing_claim','weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype',
                     'weapdetail':'weapon_details','nwound':'nb_vitcims_injured','nwoundte':'nb_perpetrators_injured',
                     'propextent_txt':'property_damage','ransomamt':'amount_ransom','hostkidoutcome_txt':'hostkidoutcome'
                    },inplace=True)

# Suppression des colonnes non utilisées pour l'analyse

##### Pour cette analyse, nous nous baserons uniquement sur valeurs principales => Nous conserverons les données "1" de crit, attacktype_txt, targtype_txt, corp, ,target, targtype ...

In [22]:
data.drop([
    'country', 'region','specificity','vicinity','doubtterr','alternative','alternative_txt', 'attacktype1',
    'attacktype2','attacktype3', 'targtype1', 'targtype2', 'targtype3', 'targsubtype1', 'targsubtype2', 'targsubtype3',
    'natlty2', 'natlty3', 'claimmode', 'claimmode2', 'claimmode3', 'weaptype1', 'weaptype1', 'weaptype2', 'weaptype3',
    'weaptype4', 'weapsubtype1', 'weapsubtype2', 'weapsubtype3', 'weapsubtype4', 'nkillus', 'nwoundus', 'nhostkidus',
    'ransomamtus', 'hostkidoutcome', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related', 'crit2', 'crit3',
    'natlty1', 'attacktype2_txt', 'targtype3_txt', 'guncertain3', 'claim2', 'claimmode2_txt', 'claim3', 'claimmode3_txt',
    'attacktype2_txt','attacktype3_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 
    'targsubtype3_txt', 'corp3','target3', 'natlty3_txt', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'guncertain2',
    'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt',
    'guncertain1','propextent','ransompaidus', 'dbsource','approxdate','extended','resolution','property','property_damage',
    'propvalue','propcomment','nhours','ndays','divert','kidhijcountry','ransom','amount_ransom','ransompaid','ransomnote',
    'hostkidoutcome','nreleased'   
], axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
df.drop([
    'country', 'region','specificity','vicinity','doubtterr','alternative','alternative_txt', 'attacktype1',
    'attacktype2','attacktype3', 'targtype1', 'targtype2', 'targtype3', 'targsubtype1', 'targsubtype2', 'targsubtype3',
    'natlty2', 'natlty3', 'claimmode', 'claimmode2', 'claimmode3', 'weaptype1', 'weaptype1', 'weaptype2', 'weaptype3',
    'weaptype4', 'weapsubtype1', 'weapsubtype2', 'weapsubtype3', 'weapsubtype4', 'nkillus', 'nwoundus', 'nhostkidus',
    'ransomamtus', 'hostkidoutcome', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related', 'crit2', 'crit3',
    'natlty1', 'attacktype2_txt', 'targtype3_txt', 'guncertain3', 'claim2', 'claimmode2_txt', 'claim3', 'claimmode3_txt',
    'attacktype2_txt','attacktype3_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2', 'natlty2_txt', 
    'targsubtype3_txt', 'corp3','target3', 'natlty3_txt', 'gname2', 'gsubname2', 'gname3', 'gsubname3', 'guncertain2',
    'weaptype2_txt', 'weapsubtype2_txt', 'weaptype3_txt', 'weapsubtype3_txt', 'weaptype4_txt', 'weapsubtype4_txt',
    'guncertain1','propextent','ransompaidus', 'dbsource','approxdate','extended','resolution','property','property_damage',
    'propvalue','propcomment','nhours','ndays','divert','kidhijcountry','ransom','amount_ransom','ransompaid','ransomnote',
    'hostkidoutcome','nreleased'  
], axis=1, inplace=True)

In [27]:
for i in df.columns:
    print([i])

['eventid']
['year']
['month']
['day']
['provstate']
['city']
['latitude']
['longitude']
['location']
['summary']
['criteria']
['multiple']
['success']
['suicide']
['attacktype']
['targtype']
['targsubtype']
['targeted_entity']
['target1']
['targeted_nationality']
['perpetrator_group']
['perpetrator_details']
['motive']
['individual']
['nperps']
['nperpcap']
['claimed']
['claim_mode']
['competing_claim']
['weapon_type']
['weapon_subtype']
['weapon_details']
['nkill']
['nkillter']
['nb_vitcims_injured']
['nb_perpetrators_injured']
['ishostkid']
['nhostkid']
['addnotes']
['scite1']
['scite2']
['scite3']


# Standardisation des pays

In [28]:
test = df.copy()

In [29]:
new_country = {
    'Czech Republic': ['Prague', 'Usti Nad Labem', 'Halickuv Brod', 'Hradec Kralove'],
    'Slovak Republic': ['Bratislava'],
    'Serbia': ['Velki Trnovac','Medvedja','Muhovac','Pozarevac','Oraovica','Cerevajka','Bujic','Banjica',
                'Bujanovac','Lucane','Belgrade', 'Bujanovac', 'Vrbas', 'Lucani', 'Bujanovac', 'Dobrosin',
                'Suva Reka', 'Podujeva','Donje Prekaze','Subotica','Batlava','Velika Reka','Loznica','Novi Sad',
                'Cacak','Zemun','Valjevo','Nis','Surcin','Veliki Trnovac','Konculj','Borovac','Vranje'],
    'Montenegro': ['Borovica','Podgorica', 'Berane', 'Podgorica', 'Kolasin','Berane','Cetinje','Podgorica','Budva'],
    'Macedonia': ['Skopje','Careva Cesma','Prilep'],
    'Slovenia': ['Ljubljana','Stari Trg'],
    'Croatia': ['Kijevo','Dubrovnik','Osijek','Benkovac','Zagreb','Pribudic','Dvor na Uni','Knin','Polaca','Sotin',
                 'Petrinja','Titova Korenica','Pakrac','Lisani','Dolac'],
    'Kosovo': ['Streoc','Dolovo','Pristina', 'Glogovac','Titova Mitrovica','Stimlje','Decani','Peje','Kosovska Mitrovica',
                'Krpimej','Donje Ljupce','Surkis','Istinic','Bogicevica','Srbica','Runik','Prizren','Kline','Gornja Klina',
                'Djakovica','Klincina','Grabanicë','Malisevo','Opterusa','Magurë','Orlat','Grabc','Ljubenic','Blace','Voksh',
                'Dulje','Tusuz','Donji Crnobreg','Komoran','Podujevë','Prekale','Kosovo Polje','Urosevac','Besinje','Kruska'],
    'Bosnia-Herzegovina': ['Sarajevo','Odzak','Banja Luka','Mostar','Kalesija','Bosanski Brod','Bijelina','Celopek'],
    'Albania': ['Vucitrn'],
    'Russia': ['Moscow','Buynaksk','Kara-Su','Novgorod','Vasyurinskaya','Sverdlovsk','Grebenskaya','Temirgoye',
              'Irkutsk'],
    'Georgia': ['Sukhumi','Meghri district','Tbilisi'],
    'Armenia': ['Yerevan','Aygepar','Kirovakan'],
    'Azerbaijan': ['Yevlakh','Nagorno-Karabakh district','Khankendi','Baganis-Airum','Chaily','Tartar district',
                  'Stepanakert','Gyandzha','Kazakh','Kazakh district','Agstafa district','Aterk'],
    'Kyrgyzstan': ['Osh'],
    'Ukraine': ['Feodosia'],
    'Estonia': ['Tallinn','Ikla'],
    'Latvia': ['Riga','Ainazi'],
    'Lithuania': ['Lavoriskes','Birzai','Salociai','Vilnius','Medininkai']
}

In [30]:
# Nombre de villes à renomer 
count = 0 

for i in new_country:
    if isinstance(new_country[i],list):
        count += len(new_country[i])

print(count)

154


In [31]:
for countries, cities in new_country.items():
    for city in cities:
        print('Il y a en ' + countries + ' toutes les villes suivantes ' + city)

Il y a en Czech Republic toutes les villes suivantes Prague
Il y a en Czech Republic toutes les villes suivantes Usti Nad Labem
Il y a en Czech Republic toutes les villes suivantes Halickuv Brod
Il y a en Czech Republic toutes les villes suivantes Hradec Kralove
Il y a en Slovak Republic toutes les villes suivantes Bratislava
Il y a en Serbia toutes les villes suivantes Velki Trnovac
Il y a en Serbia toutes les villes suivantes Medvedja
Il y a en Serbia toutes les villes suivantes Muhovac
Il y a en Serbia toutes les villes suivantes Pozarevac
Il y a en Serbia toutes les villes suivantes Oraovica
Il y a en Serbia toutes les villes suivantes Cerevajka
Il y a en Serbia toutes les villes suivantes Bujic
Il y a en Serbia toutes les villes suivantes Banjica
Il y a en Serbia toutes les villes suivantes Bujanovac
Il y a en Serbia toutes les villes suivantes Lucane
Il y a en Serbia toutes les villes suivantes Belgrade
Il y a en Serbia toutes les villes suivantes Bujanovac
Il y a en Serbia toute

In [32]:
'''for i in test['country']:
    if i == 'Yugoslavia':
        for countries, cities in new_country.items():
            for city in cities:
                test['country'] = countries   '''     

"for i in test['country']:\n    if i == 'Yugoslavia':\n        for countries, cities in new_country.items():\n            for city in cities:\n                test['country'] = countries   "

In [34]:
'''for i in test.index:
    if test['country'] == 'West Germany (FRG)':
        test.at[i, 'ifor'] = 'Germany'
    elif test['country'] == 'East Germany (GDR)':
        test.at[i, 'ifor'] = 'Germany'''
        

"for i in test.index:\n    if test['country'] == 'West Germany (FRG)':\n        test.at[i, 'ifor'] = 'Germany'\n    elif test['country'] == 'East Germany (GDR)':\n        test.at[i, 'ifor'] = 'Germany"

In [35]:
'''test[test['city'] == 'Berlin']'''

"test[test['city'] == 'Berlin']"

In [36]:
'''test[test['city'] == 'Bonn']'''

"test[test['city'] == 'Bonn']"

In [30]:
soviet_union = test[test['country'] == 'Soviet Union']
soviet_union['city'].unique()

array([], dtype=object)

In [31]:
czechoslovakia = test[test['country'] == 'Czechoslovakia']
czechoslovakia['city'].unique()

array([], dtype=object)

In [32]:
yugoslavia = test[test['country'] == 'Yugoslavia']
yugoslavia['city'].unique()

array([], dtype=object)

In [59]:
df[df['perpetrator_group'] == 'Unknown']

,eventid,year,month,day,approxdate,extended,resolution,country,region,provstate,city,latitude,longitude,location,summary,criteria,multiple,success,suicide,attacktype,targtype,targsubtype,targeted_entity,target1,targeted_nationality,perpetrator_group,perpetrator_details,motive,individual,nperps,nperpcap,claimed,claim_mode,competing_claim,weapon_type,weapon_subtype,weapon_details,nkill,nkillter,nb_vitcims_injured,nb_perpetrators_injured,property,property_damage,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,amount_ransom,ransompaid,ransomnote,hostkidoutcome,nreleased,addnotes,scite1,scite2,scite3
7,197001020002,1970,Jan,2,NaN,0,NaN,United States,North America,California,Oakland,37.791927,-122.225906,Edes Substation,1/2/1970: Unknown perpetrators detonated explo...,1,0.0,1,0,Bombing/Explosion,Utilities,Electricity,Pacific Gas & Electric Company,Edes Substation,United States,Unknown,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,Explosives,Unknown Explosive Type,NaN,0.0,0.0,0.0,0.0,1,Minor (likely < $1 million),22500.0,Three transformers were damaged.,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,"Damages were estimated to be between $20,000-$...",Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN
12,197001080001,1970,Jan,8,NaN,0,NaN,Italy,Western Europe,Lazio,Rome,41.890961,12.490069,NaN,NaN,1,0.0,1,0,Hijacking,Airports & Aircraft,Aircraft (not at an airport),Trans World Airline,Flight 802 Boeing 707,United States,Unknown,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN,Firearms,Rifle/Shotgun (non-automatic),Rifle - carbine; Pistols,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,Beirut,Beirut,0.0,NaN,NaN,NaN,Hostage(s) released by perpetrators,NaN,NaN,NaN,NaN,NaN
25,197001200001,1970,Jan,20,NaN,0,NaN,Guatemala,Central America & Caribbean,Guatemala,Guatemala City,14.622869,-90.529068,NaN,NaN,1,0.0,1,0,Assassination,Government (Diplomatic),Embassy/Consulate,British consulate,"Bodyguard, British Consul",Great Britain,Unknown,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,1.0,NaN,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,197001210001,1970,Jan,21,NaN,0,NaN,Philippines,Southeast Asia,Metropolitan Manila,Quezon City,14.674280,121.057495,NaN,NaN,1,0.0,1,0,Bombing/Explosion,Government (Diplomatic),Embassy/Consulate,NaN,JUSMAG HQ,United States,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Explosives,Vehicle,Explosive,0.0,NaN,0.0,NaN,1,Minor (likely < $1 million),5000.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,197001280001,1970,Jan,28,NaN,0,NaN,East Germany (GDR),Eastern Europe,Berlin,Berlin,52.501530,13.401851,NaN,NaN,1,0.0,1,0,Facility/Infrastructure Attack,Police,Police Security Forces/Officers,NaN,Residence of President of Penal Office,Germany,Unknown,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,Incendiary,Molotov Cocktail/Petrol Bomb,Molotov cocktail,NaN,NaN,NaN,NaN,1,Unknown,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,197001310002,1970,Jan,31,NaN,0,NaN,United States,North America,Colorado,Denver,39.758968,-104.876305,2105 Decatur Street,1/31/1970: Unknown perpetrators bombed the Pol...,1,0.0,1,0,Bombing/Explosion,Police,"Police Building (headquarters, station, school)",Denver Police Department,"Police Band Building, Denver Colorado",United States,Unknown,NaN,NaN,0,-99.0,-99.0,0.0,NaN,NaN,Explosives,Dynamite/TNT,Suspected use of dynamite,0.0,0.0,0.0,0.0,1,Minor (likely < $1 million),NaN,Restroom damaged,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Committee on Government Operations United Stat...,"Christopher Hewitt, ""Political Violence and Te...",NaN
43,197002020001,1970,Feb,2,NaN,0,NaN,United States,North America,New York,New York City,40.697132,-73.931351,Manhattan,2/2/1970: An incendiary device was discovered ...,1,0.0,0,0,Facility/Infrastructure Attack,Business,Retail/Grocery/Bakery,Mays Department Store,Mays Department Store in Manhattan,United States,Unknown,NaN,NaN,0,-99.0,-99.0

In [63]:
drop = df.copy()

In [64]:
drop.dropna(axis=0, how='any',inplace=True)

In [42]:
df.to_csv('terrorism_global.csv', sep=',', index=False, header=False, encoding='utf-8')

In [33]:
random_button.to_csv('terrorism_random_button.csv')